# Preprocessing

In [1]:
import pandas as pd

In [2]:
reviews = pd.read_csv('data/raw/reviews.csv', parse_dates=["date"])
reviews = reviews.applymap(lambda x: x.lower() if isinstance(x, str) else x)
reviews.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03
1,125646,_dirty_,2017-12-21,,4.50,4.50,4.50,4.50,4.50,4.50
2,125646,cjdubya,2017-12-21,,4.75,4.75,4.75,4.75,4.75,4.75
3,125646,gratefulbeerguy,2017-12-20,0% 16 oz can. funny story: as i finally wal...,4.75,4.75,4.50,4.50,4.50,4.58
4,125646,lukegude,2017-12-20,classic th neipa. overflowing head and bouq...,4.25,4.50,4.25,4.25,4.25,4.31


In [3]:
# load breweries and transform the 'types' into a one-hot encoding representation
breweries = pd.read_csv('data/raw/breweries.csv', index_col="id")
breweries["brewery_type_bar"] = breweries["types"].apply(lambda x: int("bar" in x.lower()))
breweries["brewery_type_beer_to_go"] = breweries["types"].apply(lambda x: int("to-go" in x.lower()))
breweries["brewery_type_eatery"] = breweries["types"].apply(lambda x: int("eatery" in x.lower()))
breweries["brewery_type_homebrew"] = breweries["types"].apply(lambda x: int("homebrew" in x.lower()))
breweries["brewery_type_brewery"] = breweries["types"].apply(lambda x: int("brewery" in x.lower()))
breweries["brewery_type_store"] = breweries["types"].apply(lambda x: int("store" in x.lower()))
breweries = breweries.drop(columns=["types", "notes"])
breweries = breweries.rename(columns={"name":"brewery_name"})
breweries = breweries.applymap(lambda x: x.lower() if isinstance(x, str) else x)
breweries.head()

,brewery_name,city,state,country,brewery_type_bar,brewery_type_beer_to_go,brewery_type_eatery,brewery_type_homebrew,brewery_type_brewery,brewery_type_store
id,,,,,,,,,,
19730,brouwerij danny,erpe-mere,NaN,be,0,0,0,0,1,0
32541,coachella valley brewing co,thousand palms,ca,us,1,1,0,0,1,0
44736,beef 'o' brady's,plant city,fl,us,1,0,1,0,0,0
23372,broadway wine merchant,oklahoma city,ok,us,0,0,0,0,0,1
35328,brighton beer dispensary (duplicate),brighton,gb2,gb,1,0,1,0,0,0


In [4]:
beers = pd.read_csv('data/raw/beers.csv')
beers = beers.drop(columns=["state", "country", "notes"])
beers = beers.rename(columns={"style":"style_detailed"})
beers = beers.applymap(lambda x: x.lower() if isinstance(x, str) else x)
beers["retired"] = beers["retired"].apply(lambda x: int(x=='t'))
beers.head()

,id,name,brewery_id,style_detailed,availability,abv,retired
0,202522,olde cogitator,2199,english oatmeal stout,rotating,7.3,0
1,82352,konrads stout russian imperial stout,18604,russian imperial stout,rotating,10.4,0
2,214879,scottish right,44306,scottish ale,year-round,4.0,1
3,320009,megameow imperial stout,4378,american imperial stout,winter,8.7,0
4,246438,peaches-n-cream,44617,american cream ale,rotating,5.1,0


In [5]:
aromas = pd.read_csv('data/raw/beer_data_set.csv', index_col="key")
aromas = aromas.applymap(lambda x: x.lower() if isinstance(x, str) else x)
aromas = aromas.rename(lambda x: "_".join(x.lower().split()), axis=1)
aromas = aromas.drop(columns=["style_key"])
aromas.head()

,name,style,brewery,description,abv,rating_source,min_ibu,max_ibu,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
key,,,,,,,,,,,,,,,,,,,
251,amber,altbier,alaskan brewing co.,"notes:richly malty and long on the palate, wit...",5.3,3.65,25,50,13,32,9,47,74,33,0,33,57,8,111
252,double bag,altbier,long trail brewing co.,"notes:this malty, full-bodied double alt is al...",7.2,3.90,25,50,12,57,18,33,55,16,0,24,35,12,84
253,long trail ale,altbier,long trail brewing co.,notes:long trail ale is a full-bodied amber al...,5.0,3.58,25,50,14,37,6,42,43,11,0,10,54,4,62
254,doppelsticke,altbier,uerige obergärige hausbrauerei,notes:,8.5,4.15,25,50,13,55,31,47,101,18,1,49,40,16,119
255,scurry,altbier,off color brewing,notes:just cause it's dark and german doesn't ...,5.3,3.67,25,50,21,69,10,63,120,14,0,19,36,15,218


In [6]:
# merge beers with their respective breweries, dropping element which cannot be matched
beers = beers.merge(breweries, left_on="brewery_id", right_index=True, how="inner")
beers = beers.drop(columns=["brewery_id"])
print(beers.shape)
beers.head()

(358873, 16)


,id,name,style_detailed,availability,abv,retired,brewery_name,city,state,country,brewery_type_bar,brewery_type_beer_to_go,brewery_type_eatery,brewery_type_homebrew,brewery_type_brewery,brewery_type_store
0,202522,olde cogitator,english oatmeal stout,rotating,7.3,0,main street brewery,pleasanton,ca,us,1,0,1,0,1,0
43816,8677,red leaf strong ale,american strong ale,year-round,8.6,0,main street brewery,pleasanton,ca,us,1,0,1,0,1,0
46154,8129,hefeweisen,german hefeweizen,year-round,NaN,1,main street brewery,pleasanton,ca,us,1,0,1,0,1,0
54879,202536,bikini bottom,american lager,rotating,4.9,0,main street brewery,pleasanton,ca,us,1,0,1,0,1,0
96377,33121,fog lifter ipa,american ipa,rotating,NaN,1,main street brewery,pleasanton,ca,us,1,0,1,0,1,0


In [7]:
beers = beers.merge(aromas, left_on=["name", "brewery_name", "abv"], right_on=["name", "brewery", "abv"], how="inner")
beers = beers[
    [
        "id", "name", "abv",
        "style", "style_detailed",
        "min_ibu", "max_ibu", "alcohol", "astringency", "bitter", "body", "fruits", "hoppy", "malty", "salty", "sour", "spices", "sweet",
        "brewery_name", "city", "state", "country", 
            "brewery_type_bar", "brewery_type_beer_to_go", "brewery_type_eatery", "brewery_type_homebrew", "brewery_type_brewery", "brewery_type_store",
        "availability", "retired", "rating_source"
        
    ]
]
print(beers.shape)
beers.head()

(4135, 31)


,id,name,abv,style,style_detailed,min_ibu,max_ibu,alcohol,astringency,bitter,...,country,brewery_type_bar,brewery_type_beer_to_go,brewery_type_eatery,brewery_type_homebrew,brewery_type_brewery,brewery_type_store,availability,retired,rating_source
0,64620,the citizen,6.00,california common / steam beer,california common / steam beer,35,45,7,18,45,...,us,1,1,0,0,1,0,year-round,0,3.74
1,82938,arthur,4.20,mild ale - english pale,english pale mild ale,15,25,0,4,16,...,ca,0,1,0,0,1,0,rotating,0,3.64
2,8107,medalla light,4.00,lager - light,american light lager,8,12,10,15,9,...,pr,0,0,0,0,1,0,year-round,0,2.82
3,3940,imperial,4.50,lager - adjunct,american adjunct lager,8,18,4,34,22,...,cr,0,0,0,0,1,0,year-round,0,2.80
4,34877,collaboration not litigation ale,8.72,strong ale - belgian dark,belgian strong dark ale,25,50,24,10,11,...,us,1,1,1,0,1,0,rotating,0,3.93


In [8]:
print(reviews.shape)
reviews = reviews[reviews["beer_id"].isin(beers.index)]
print(reviews.shape)

(9073128, 10)
(1744151, 10)


In [9]:
for col in ["look", "smell", "taste", "feel", "overall", "score"]:
    beers["rating_reviews_"+col] = beers["id"].apply(lambda x: reviews[reviews["beer_id"] == x][col].median(skipna=True))
beers.head()

,id,name,abv,style,style_detailed,min_ibu,max_ibu,alcohol,astringency,bitter,...,brewery_type_store,availability,retired,rating_source,rating_reviews_look,rating_reviews_smell,rating_reviews_taste,rating_reviews_feel,rating_reviews_overall,rating_reviews_score
0,64620,the citizen,6.00,california common / steam beer,california common / steam beer,35,45,7,18,45,...,0,year-round,0,3.74,NaN,NaN,NaN,NaN,NaN,NaN
1,82938,arthur,4.20,mild ale - english pale,english pale mild ale,15,25,0,4,16,...,0,rotating,0,3.64,NaN,NaN,NaN,NaN,NaN,NaN
2,8107,medalla light,4.00,lager - light,american light lager,8,12,10,15,9,...,0,year-round,0,2.82,NaN,NaN,NaN,NaN,NaN,NaN
3,3940,imperial,4.50,lager - adjunct,american adjunct lager,8,18,4,34,22,...,0,year-round,0,2.80,3.0,2.75,3.0,3.0,3.0,2.8
4,34877,collaboration not litigation ale,8.72,strong ale - belgian dark,belgian strong dark ale,25,50,24,10,11,...,0,rotating,0,3.93,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
beers.to_csv("data/processed/beers.csv", index=False)

In [11]:
reviews.to_csv("data/processed/reviews.csv", index=False)

: 